In [1]:
import numpy as np
from tqdm import tqdm
from utils import read_data, read_query, read_output, knn, distances

In [2]:
data_path = '../data/dummy-data.bin'
query_path = '../data/dummy-queries.bin'

output_path_1 = '../data/dummy-output-seqscan.bin'
output_path_2 = '../data/dummy-output-seqscanrange.bin'

In [6]:
# read data, query and output1, output2
data = read_data(data_path)
query = read_query(query_path) 
output1 = read_output(output_path_1, query.shape[0]).astype(int)
output2 = read_output(output_path_2, query.shape[0]).astype(int)
# calculate distances of outputs

print("="*10)
print("data shape", data.shape)
print("query shape", query.shape)
print("="*10)
print("output1 shape", output1.shape)
print("output2 shape", output2.shape)
print("="*4,"KNN","="*4)


# True knn
output_true, dist, dist_true = knn(data, query, 100)
dist_output1 = distances(output1, dist)
#dist_output2 = distances(output2, dist)

print("="*10)
print("output_true shape", output_true.shape)
print("dist shape", dist.shape)
print("="*10)
print("dist_true shape", dist_true.shape)
print("dist_output1 shape", dist_output1.shape)


data shape (10000, 102)
query shape (100, 104)
output1 shape (100, 100)
output2 shape (100, 100)
==== KNN ====


100%|██████████| 100/100 [00:01<00:00, 67.94it/s]

output_true shape (100, 100)
dist shape (10000, 100)
dist_true shape (100, 100)
dist_output1 shape (100, 100)


In [14]:
a = np.array([5,10,20,30,np.inf])
a_idx = np.array([0,1,2,3,5])

b = np.array([10,20,30,40,40])
b_idx = np.array([1,2,3,4,4])

# replace inf with max value and return the index of the max value
a[a == np.inf] = np.max(a[np.isfinite(a)])
a.argmax()



3

In [17]:
def _recall(idx1, idx2, dist1, dist2):
    """
    idx1, idx2: 2d array of shape (n_queries, k) 
    dist1, dist2: 2d array of shape (n_queries, k)
    idx1 treated as true knn
    """
    
    # replace inf with max value and return the index of the max value and max value 
    idx1[idx1 == np.inf] = np.max(idx1[np.isfinite(idx1)])
    id_max = idx1.argmax()
    _max = idx1.max()
    
    print("id_max": id_max)
    print("_max": _max)
    
    # remove dublicates from idx2 and keep their indexes to remove the same indexes from dist2
    idx2_unique, idx2_idx = np.unique(idx2, return_index=True)
    dist2_unique = dist2[idx2_idx]
    # remove elemnts that are > _max from idx2 and dist2
    idx2_unique = idx2_unique[idx2_unique < _max]
    dist2_unique = dist2_unique[idx2_unique < _max]
    
    # calculate recall: 
    recall = idx2_unique.shape[0] / id_max 
    return recall
    

In [18]:
_recall(a_idx,b_idx,a,b)

1.0

In [6]:
def recall(output_true, output):
    return np.mean([np.intersect1d(output_true[i], output[i]).shape[0] for i in range(output.shape[0])])

9658

In [15]:
# number of values in output2 that are > than 10000:
(output2 > 10000).sum()

2300

In [16]:
c = 0
# get indices of output 2 which are > 10000
for i in range(output2.shape[0]):
    for j in range(output2.shape[1]):
        if output2[i][j] > 10000:
            print(i, j, output2[i][j])
            c += 1
print(c)


3 0 2885787070
3 1 2885787070
3 2 2885787070
3 3 2885787070
3 4 2885787070
3 5 2885787070
3 6 2885787070
3 7 2885787070
3 8 2885787070
3 9 2885787070
3 10 2885787070
3 11 2885787070
3 12 2885787070
3 13 2885787070
3 14 2885787070
3 15 2885787070
3 16 2885787070
3 17 2885787070
3 18 2885787070
3 19 2885787070
3 20 2885787070
3 21 2885787070
3 22 2885787070
3 23 2885787070
3 24 2885787070
3 25 2885787070
3 26 2885787070
3 27 2885787070
3 28 2885787070
3 29 2885787070
3 30 2885787070
3 31 2885787070
3 32 2885787070
3 33 2885787070
3 34 2885787070
3 35 2885787070
3 36 2885787070
3 37 2885787070
3 38 2885787070
3 39 2885787070
3 40 2885787070
3 41 2885787070
3 42 2885787070
3 43 2885787070
3 44 2885787070
3 45 2885787070
3 46 2885787070
3 47 2885787070
3 48 2885787070
3 49 2885787070
3 50 2885787070
3 51 2885787070
3 52 2885787070
3 53 2885787070
3 54 2885787070
3 55 2885787070
3 56 2885787070
3 57 2885787070
3 58 2885787070
3 59 2885787070
3 60 2885787070
3 61 2885787070
3 62 2885787070
3 